## 운동 동작 분류 AI 경진대회

In [6]:
import pandas as pd

In [45]:
X_train = pd.read_csv('./exercise/train_features.csv')
y_train = pd.read_csv('./exercise/train_labels.csv')
X_test = pd.read_csv('./exercise/test_features.csv')
sample = pd.read_csv('./exercise/sample_submission.csv')

In [12]:
X_train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [11]:
y_train

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [25]:
len(y_train.label.unique())
y_train.label.unique()

array([37, 26,  3, 39, 28,  6, 60, 35, 34, 30, 59, 15, 40, 50, 48,  8, 54,
       16,  4, 58, 21, 18, 51, 25, 24, 17, 49, 47, 27, 36,  2, 38, 14, 10,
       55, 57,  0,  1, 46, 43, 52, 53,  5, 56, 23, 44, 42, 19, 32, 45, 29,
       31, 11,  7, 41, 12,  9, 20, 13, 22, 33], dtype=int64)

In [17]:
# id당 운동 600회
X_train.shape[0]/len(X_train.id.unique())

600.0

In [21]:
# 결측치 없음
X_train.isnull().sum()

id       0
time     0
acc_x    0
acc_y    0
acc_z    0
gy_x     0
gy_y     0
gy_z     0
dtype: int64

In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875000 entries, 0 to 1874999
Data columns (total 8 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id      int64  
 1   time    int64  
 2   acc_x   float64
 3   acc_y   float64
 4   acc_z   float64
 5   gy_x    float64
 6   gy_y    float64
 7   gy_z    float64
dtypes: float64(6), int64(2)
memory usage: 114.4 MB


In [29]:
len(X_test.id.unique())

782

In [27]:
sample.shape

(782, 62)

In [34]:
X_test.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043
3,3125,3,-0.351750,-0.239870,-0.193053,23.686172,88.608721,13.449771
4,3125,4,-0.312934,-0.123762,-0.318621,20.410071,85.327707,13.884912


In [105]:
X_train_gp = X_train.groupby('id').agg(['max','min','mean'])[['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']]
X_train_gp

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
0     1.344268  0.591940  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1     1.234020 -2.156208 -0.766580  0.700065 -1.295598 -0.317258  0.888661   
2     1.219836 -1.142847  0.039836  0.650645 -0.690990 -0.082403  1.332992   
3    -0.622250 -1.417751 -0.887702  0.283721 -0.540827 -0.087668  0.598814   
4     0.599720 -2.429109 -0.659018  1.724782 -2.055076 -0.337067  2.678034   
...        ...       ...       ...       ...       ...       ...       ...   
3120  0.390798 -1.624711 -0.300454  0.168070 -1.289257 -0.669209  0.849966   
3121 -0.446650 -1.575455 -0.974298  0.117965 -0.609743 -0.233373  0.218621   
3122  0.744666 -2.578974 -1.114246  1.268138 -2.036646 -0.362196  0.803950   
3123  0.915846 -0.929133 -0.111333  1.473727  0.272406  0.880362  0.382916   
3124  0.538809 -1.013813 -0.434048 -0.193247 -0.952761 -0.623010  0.543977   

                                gy_x                               gy_y  \
           min      mean         max         min       mean         max   
id                                                                        
0    -0.786336 -0.370422   31.644123  -46.254836  -1.865269   69.847244   
1    -1.019531 -0.004223  286.624363 -325.328531  11.071600  389.608060   
2     0.073846  0.626012   73.525082 -164.779067  -8.472951  297.320834   
3    -0.257124  0.227357  192.765368  -69.419166  17.744167  159.083788   
4    -1.250483  0.202758  613.972600 -769.076518  -4.819638  284.952954   
...        ...       ...         ...         ...        ...         ...   
3120 -0.551185  0.335934  148.152235 -194.369980  -5.382982  134.585268   
3121 -0.403201 -0.073771  155.958950 -119.639981  11.394976   53.341929   
3122 -0.459267  0.241518  400.662632 -731.029790  -3.821330  388.749570   
3123 -0.646863 -0.122757  168.716922 -208.206497  -6.223759  319.723456   
3124 -0.380657  0.226848   52.957569  -58.732727  -1.638705   90.173541   

                                   gy_z                        
             min       mean         max         min      mean  
id                                                             
0     -85.887677  -3.359506   55.953827  -79.930029  1.182107  
1    -315.096003   1.740475  340.170199 -270.980823  1.393294  
2    -249.953944   0.597877   55.642836  -44.192071  3.053291  
3     -82.537304   4.800931   56.456908  -85.600536 -5.869898  
4    -243.909948   9.651713  221.015193 -270.581913  4.453382  
...          ...        ...         ...         ...       ...  
3120 -190.818371  -4.902798  121.958427  -79.392292 -0.054026  
3121  -33.757220   3.786842   57.349878  -39.777626 -2.792238  
3122 -380.819269  10.172169  453.943910 -247.908573 -1.722830  
3123 -148.431769 -11.608354  310.558507 -206.580638 -5.930252  
3124 -134.268304  -1.226870   80.302840 -100.845205 -3.246825  

[3125 rows x 18 columns]

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
y_train = y_train.label

In [69]:
dt_clf =DecisionTreeClassifier(random_state=1)

In [71]:
dt_clf.fit(X_train_gp, y_train)

DecisionTreeClassifier(random_state=1)

In [72]:
dt_clf.get_depth()

37

In [106]:
dt_clf =DecisionTreeClassifier(random_state=1)

In [107]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth':list(range(1,20,2)),'min_samples_split':list(range(2,20,2))}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(X_train_gp, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.5968
{'max_depth': 11, 'min_samples_split': 18}


[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:   31.7s finished


In [86]:
y_train.head()

0    37
1    26
2     3
3    26
4    26
Name: label, dtype: int64

In [87]:
def label(x):
    return y_train[x]

In [89]:
label(0)

37

In [90]:
X_train['label']=X_train.id.apply(label)

In [92]:
X_train[X_train.id == 2].label.head()

1200    3
1201    3
1202    3
1203    3
1204    3
Name: label, dtype: int64

In [94]:
X_train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,label
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,37
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,37
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,37
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,37
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,37


In [97]:
dt_clf =DecisionTreeClassifier(random_state=1)

In [98]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth':list(range(10,20,2))}

grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(X_train.iloc[:,:-1], X_train.label)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 12.4min finished


0.3501525333333334
{'max_depth': 10}


In [110]:
X_test_gp = X_test.groupby('id').agg(['max','min','mean'])[['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']]
X_test_gp

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
3125 -0.275446 -1.564000 -1.018731  0.228040 -0.470937 -0.019574  0.286182   
3126  0.627571 -1.929033 -0.522843  1.708743 -0.200678  0.612161  0.671876   
3127  2.972063 -0.792916  0.506947  1.941820  0.219008  0.903819  0.644154   
3128  0.337281 -1.045889 -0.577603 -0.258476 -1.294482 -0.610557  0.702574   
3129  0.015642 -2.153047 -0.738640  1.562602 -0.860883  0.182535  1.037876   
...        ...       ...       ...       ...       ...       ...       ...   
3902  0.427159 -2.050254 -0.907299  3.057501 -1.414874  0.488264  0.237183   
3903  1.659451 -1.709527 -0.608731  1.549890 -1.247963  0.371269  0.713875   
3904 -0.085249 -2.124959 -0.753193  1.236138 -0.443533  0.199782  0.939270   
3905  1.438345  0.536568  0.958903  0.076427 -0.580191 -0.320975  0.097833   
3906  0.224194 -1.534616 -0.781752  0.144179 -0.958351 -0.369244  1.033111   

                                gy_x                               gy_y  \
           min      mean         max         min       mean         max   
id                                                                        
3125 -0.573836 -0.232259   56.953059  -50.429364  -3.185128   96.185341   
3126 -1.212052 -0.071092  132.830402 -273.572486 -17.741526  241.240196   
3127 -0.484614 -0.141098  160.426058  -99.799971  -2.179745   74.530763   
3128 -0.469924  0.031486  119.527887 -229.072919   0.808579  118.268797   
3129 -0.631258  0.401302  366.167357 -345.447240  -1.431538  226.728939   
...        ...       ...         ...         ...        ...         ...   
3902 -1.517656 -0.238160  749.793156 -677.830890   5.218293  220.936927   
3903 -0.996954 -0.106845  413.572371 -265.501904   6.350148  225.268834   
3904 -0.332901  0.311769  180.002544 -214.831199  -0.761053  129.420821   
3905 -0.774209 -0.356234   66.788750  -63.337676   2.559009  101.379591   
3906 -0.539595  0.141230  344.843462 -161.399629 -11.480833  186.272547   

                                  gy_z                         
             min      mean         max         min       mean  
id                                                             
3125  -81.607713  2.731872   49.981455  -35.446915  -2.000683  
3126  -97.100707  6.974772  169.417650 -147.597574  -3.604579  
3127  -98.420987 -2.251452   97.211730 -154.477074  -0.393175  
3128 -168.031080 -2.984969  167.860762 -117.297766  -0.024318  
3129 -223.475411  1.442366  138.130133 -125.598600   5.745498  
...          ...       ...         ...         ...        ...  
3902 -212.399091  1.218743  214.192019 -241.419881  -0.429678  
3903 -205.407503  3.987078  253.689077 -164.337764  13.272141  
3904 -142.559610 -2.325664  251.344358 -150.012379   1.935083  
3905  -83.950263 -1.653229   58.041427  -51.905231   0.932313  
3906 -112.020378 -3.664275  101.996940 -190.358982  -7.197783  

[782 rows x 18 columns]

In [111]:
best_clf = grid_cv.best_estimator_
pred = best_clf.predict_proba(X_test_gp)
pred

array([[0.        , 0.        , 0.        , ..., 0.15384615, 0.        ,
        0.07692308],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.18181818, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.004     , 0.        , 0.        , ..., 0.        , 0.        ,
        0.002     ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.004     , 0.        , 0.        , ..., 0.        , 0.        ,
        0.002     ]])

In [112]:
sample.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
sample.iloc[:,1:]=pred

In [117]:
sample.to_csv('answer.csv',index=False)